# Práctica Dataset Fútbol. Realizado por Daniel Tomé Gordo 
## Máster en Data Science para Finanzas de CUNEF

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

## Primera pregunta: ¿ En que se diferencian las acciones de las transformaciones en SPARK? ¿Podrías ponerme un par de ejemplo de cada una de ellas y describir que hacen, para que sirve?

Las transformaciones en un RDD existente, devuelven otro RDD. Por ejemplo, si se aplicase un filtro. En este caso se podría filtrar por distintos campos como el equipo local, visitante, la temporada o los goles. El resultado de aplicar el filtro al RDD es otro RDD al que se podrá acceder. El acceso a ese RDD se debe realizar mediante una acción, como puede ser un _take_ para sacar x registros, o un _count_ para contar el total de registros.
Con estas acciones ya se puede operar y acceder a ellas. Durante esta práctica se realizarán accesos (acciones) en las salidas de los resultados, a las que previamente se han realizado transformaciones. También cabe decir que las transformaciones son de carácter _lazy_.

## Segunda tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
archivo = "Datos/DataSetPartidos.txt"
data = sc.textFile(archivo)

Se realiza un _take_ para ver como es la estructura de los datos. Parece ser que el primer dato es un identificador, el segundo la temporada en la que el partido se juega, el tercero podría ser la jornada en la que se juega el partido, siendo el cuarto y el quinto datos los equipos que jugaron el partido de local y visitante respectivamente. El sexto y séptimo dato son los goles del partido, local y visitante, respectivamente. El octavo es la fecha del partido y el último podría ser la asistencia (no está claro).
Con el _count_ hallamos los registros totales, 34.518 partidos en este caso

In [5]:
data.count()

34518

In [4]:
data.take(1)

['1::1970-71::1::Athletic Club::Barcelona::1::1::12/09/1970::21942000.0']

Para parsear los datos, se observa que cada variable está dividida por **::**, por lo que se divide cada registro por esos caracteres.

In [3]:
datos_parseados = data.map(lambda x: x.split("::"))

In [4]:
datos_parseados.take(1)

[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

## Tercera pregunta: Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

En primer lugar se deberán filtrar todos los partidos en los que el Sporting de Gijón juega en casa. Se coge el cuarto elemento de los datos parseados. Una vez filtrados los registros, se coge el sexto elemento (los goles del equipo local) y se suman todos. Así ya se habrían conseguido los goles que el Sporting ha marcado en El Molinón durante las temporadas contenidas en la base de datos. El proceso es el mismo para los goles que el Sporting ha marcado fuera de casa.

Finalmente se "imprimen" los goles del Sporting en esta temporada, sumando ambos resultados.

In [5]:
sporting_local = datos_parseados.filter(lambda x: x[3] == "Sporting de Gijon") # Se filtra por el equipo local (Sporting)
goles_local = sporting_local.map(lambda x: int(x[5])).sum() # Goles marcados en El Molinón

In [6]:
sporting_visitante = datos_parseados.filter(lambda x: x[4] == "Sporting de Gijon") # Se filtra por el equipo visitante
goles_visitante = sporting_visitante.map(lambda x: int(x[6])).sum() # Goles marcados por el Sporting fuera de casa

In [7]:
print ("El Real Sporting de Gijón ha marcado",goles_local + goles_visitante, "goles a lo largo de las 45 temporadas")

El Real Sporting de Gijón ha marcado 2042 goles a lo largo de las 45 temporadas


## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

En primer lugar, se calculan todos los goles que se han marcado en los 34.518 partidos. Para ello, se hallan los goles marcados en los partidos y se guardan en el formato _temporada;goles_. En el último paso, se reducen los registros sumando todos los goles de cada temporada. Para hallar la temporada en la que más goles se han conseguido, se realiza un _takeOrdered_ sacando un registro (el que más valor tenga). Así la salida, será la temporada en la que más goles se metieron.
Finalmente, se imprime la información.

In [8]:
temporada_goles = datos_parseados.map(lambda x: (x[1], int(x[5]) + int(x[6]))).reduceByKey(lambda a, b: a + b)

In [9]:
temporada_mas_goles = temporada_goles.takeOrdered(1, lambda x: -x[1])

In [10]:
print("La temporada con más goles fue la", temporada_mas_goles[0][0], "y se marcaron", temporada_mas_goles[0][1], "goles")

La temporada con más goles fue la 2012-13 y se marcaron 2294 goles


## Quinta pregunta: ¿Cúal es el equipo y tiene el record de más goles como local? ¿ y cómo visitante?

En primer lugar se sacarán todos los equipos locales de todos los partidos, y en un formato de clave (siendo esta el equipo) - valor (siendo este el número de goles). Posteriormente, se agruparán y se sumarán todos los valores. De esta manera, el que tenga un valor máximo será el equipo que más goles ha metido de local. Para el cálculo del visitante, se realizará el mismo proceso.

En el caso del visitante tanto el Real Madrid como el Barcelona han anotado 1.296 goles a lo largo de estas temporadas

In [92]:
goles_totales_local = datos_parseados.map(lambda x: (x[3], int(x[5]))).reduceByKey(lambda a, b: a + b)
mas_gol_local = goles_totales_local.takeOrdered(5, lambda x: -x[1])

In [93]:
print('El equipo con más goles como local a lo largo de estas temporadas es el:', mas_gol_local[0][0], 'con', mas_gol_local[0][1], 'goles')

El equipo con más goles como local a lo largo de estas temporadas es el: Real Madrid con 2054 goles


In [94]:
goles_totales_visitante = datos_parseados.map(lambda x: (x[4], int(x[6]))).reduceByKey(lambda a, b: a + b)
mas_gol_visitante = goles_totales_visitante.takeOrdered(5, lambda x: -x[1])
mas_gol_visitante

[('Real Madrid', 1296),
 ('Barcelona', 1296),
 ('Atletico de Madrid', 988),
 ('Valencia', 919),
 ('Betis', 853)]

In [95]:
print('El equipo con más goles como visitante a lo largo de estas temporadas es el:', mas_gol_visitante[0][0], 'con', mas_gol_visitante[0][1], 'goles')

El equipo con más goles como visitante a lo largo de estas temporadas es el: Real Madrid con 1296 goles


## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

Aprovechando el cálculo de los goles por temporada del apartado 4, se accede a los 4 primeros números de la clave (el año) y al valor (los goles) y se crea un nuevo RDD de formato (año, goles). Luego se divide entre 10 y se vuelve a multiplicar (considerándolo como entero) teniendo así la década. Se reduce por clave y se ordenan las 5 décadas existentes en el dataset.

In [96]:
goles_año = temporada_goles.map(lambda x: (x[0][0:4], x[1])) # Se consigue (año, goles) -> p.ej (1979, 3)
goles_decada = goles_año.map(lambda x: (int(x[0])//10 * 10, x[1]))
total_goles_decada = goles_decada.reduceByKey(lambda a, b: a + b)
goles_decada = total_goles_decada.takeOrdered(5, lambda x: -x[1])

In [97]:
print('La década con más goles fue la de', goles_decada[0][0], 'con', goles_decada[0][1], 'goles', '\n'
     'La segunda década con más goles fue la de', goles_decada[1][0], 'con', goles_decada[1][1], 'goles', '\n'
     'La tercera década con más goles fue la de', goles_decada[2][0], 'con', goles_decada[2][1], 'goles')

La década con más goles fue la de 2000 con 20759 goles 
La segunda década con más goles fue la de 1990 con 19426 goles 
La tercera década con más goles fue la de 1980 con 17351 goles


##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

Se sacan todos los equipos y temporadas (***temp_equipo***) para luego sumar las presencias de cada equipo (***equipo_numero_temp***)

In [ ]:
temp_equipo = datos_parseados.map(lambda x: (x[1], x[3])).distinct()
equipo_numero_temp = temp_equipo.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)

Se filtran todos los equipos que han estado menos de 10 temporadas. En este paso se detecta que no todos los partidos son de Primera División. De esta manera el ejercicio se hará con los equipos que no han aparecido en Primera o Segunda División más de 10 veces. Existen 58 equipos. Nos quedamos sólo con el nombre de los equipos (***equipos_menos10***). Luego se crea una lista con esos equipos (***equipos_menos10_temp***)

In [36]:
equipos_menos10 = equipo_numero_temp.filter(lambda x: x[1] < 10).map(lambda x: x[0])
equipos_menos10_temp = equipos_menos10.collect()

Se recuperan los datos de los equipos locales y visitantes originales. Busca en los registros cada nombre que está en la lista creada anteriormente (***equipos_menos10_temp***)

In [38]:
equipos_menos10_local = (datos_parseados.filter(lambda x: x[3] in equipos_menos10_temp))
equipos_menos10_visit = (datos_parseados.filter(lambda x: x[4] in equipos_menos10_temp))

Se calculan las victorias de cada equipo tanto jugando de local, como jugando de visitante. Para después contar las victorias y sumar agrupar por clave, es decir, por equipo.

In [39]:
victorias_equipos_menos10_local = (equipos_menos10_local.filter(lambda x: x[5] > x[6]))
victorias_equipos_menos10_visit = (equipos_menos10_visit.filter(lambda x: x[5] < x[6]))

In [40]:
victorias_locales = victorias_equipos_menos10_local.map(lambda x: (x[3], 1)).reduceByKey(lambda a, b: a + b)
victorias_visitantes = victorias_equipos_menos10_visit.map(lambda x: (x[4], 1)).reduceByKey(lambda a, b: a + b)

Se suman las victorias locales y visitantes para luego agrupar por clave (equipo) y se suman los valores de estos. Así ya se podría hacer la media de las victorias de cada equipo.

In [41]:
total_victorias = victorias_locales + victorias_visitantes
victorias_equipo = total_victorias.groupByKey().mapValues(sum)
media = victorias_equipo.map(lambda x: int(x[1])).mean()

In [111]:
print('La media de victorias de los equipos que han estado menos de 10 temporadas en Primera y/o Segunda es de', round(media, 2))

La media de victorias de los equipos que han estado menos de 10 temporadas en Primera y/o Segunda es de 46.02


##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

Se sacarán todos los goles que se han marcado como visitante en los partidos y luego se realizará la media de estos.

In [23]:
total_gol_visitante = datos_parseados.map(lambda x: int(x[6])).sum()
media_goles_visitante = (total_gol_visitante/data.count())
print('Un equipo visitante marca de media', round(media_goles_visitante, 3), 'goles por partido')

Un equipo visitante marca de media 0.928 goles por partido
